In [1]:
import pandas as pd
import codecs
import nltk
from nltk.tokenize import word_tokenize
import numpy as np
from collections import Counter

In [2]:
newspapers = pd.read_csv("C:/Users/jw156/Ironhack/vaccine/vaccine_sentiment/english/newspapers.csv", index_col=0)

In [3]:
newspapers.columns

Index(['hrefs', 'title', 'new_title', 'id', 'date', 'newspaper_name',
       'vaccine'],
      dtype='object')

In [4]:
newspapers.new_title =  newspapers.new_title.apply(str.rstrip)

In [5]:
newspapers = newspapers.reset_index(drop=True)

In [6]:
newspapers.title.drop_duplicates(inplace=True)

# POS Determination of words

In [7]:
newspapers = newspapers.reset_index(drop=True)

In [8]:
import nltk
from nltk.stem import WordNetLemmatizer 

In [9]:
from nltk.corpus import wordnet as wn
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag, word_tokenize


In [10]:
from nltk.corpus import wordnet as wn

def pos_headline(headline):
    new_headline = headline.split(' ')
    word_list = []
    for word in new_headline:
        if word == None or word.isalpha()==False:
            continue
        else:
            text = word_tokenize(word)
            postition_tag = nltk.pos_tag(text)
            word_list.append(postition_tag)
    return word_list

def is_proper_noun(tag):
    return tag in ['NNP', 'NNPS']

def is_noun(tag):
    return tag in ['NN', 'NNS']


def is_verb(tag):
    return tag in ['VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ']


def is_adverb(tag):
    return tag in ['RB', 'RBR', 'RBS']


def is_adjective(tag):
    return tag in ['JJ', 'JJR', 'JJS']



def penn_to_wn(headlines):
    headline_list = pos_headline(headlines)
    tagged_list = []
    for tag in headline_list:
        tag = list(tag[0])
        if is_adjective(tag[1]):
            tag[1] = (wn.ADJ)
            tagged_list.append(tag)
        elif is_noun(tag[1]):
            tag[1] =(wn.NOUN)
            tagged_list.append(tag)
        elif is_adverb(tag[1]):
            tag[1] =(wn.ADV)
            tagged_list.append(tag)
        elif is_verb(tag[1]):
            tag[1] =(wn.VERB)
            tagged_list.append(tag)
        elif is_proper_noun(tag[1]):
            tag[1] =('NNP')
            tagged_list.append(tag)
    return tagged_list

def lemming_words(headline):
    if len(headline) == 0:
        return 0
    headline_list = penn_to_wn(headline)    
    new_string = []
    lemmatizer = WordNetLemmatizer()
    for word in headline_list:
        if word[1] == 'NNP':
            continue
        else:
            word[0] = (lemmatizer.lemmatize(word[0], word[1]))
            new_string.append(word)

    return new_string

In [11]:
x = newspapers.title[1]
lemming_words(x)


[['Scepticism', 'n'],
 ['Oxford', 'n'],
 ['vaccine', 'n'],
 ['threatens', 'n'],
 ['immunisation', 'n'],
 ['push', 'n']]

In [12]:
newspapers['pos_headline'] = newspapers['title'].apply(pos_headline)

In [13]:
newspapers['lem_pos_headline'] = newspapers['title'].apply(lambda x: lemming_words(x))

In [14]:
newspapers['pos_headline'][1]

[[('Scepticism', 'NN')],
 [('over', 'IN')],
 [('Oxford', 'NN')],
 [('vaccine', 'NN')],
 [('threatens', 'NNS')],
 [('immunisation', 'NN')],
 [('push', 'NN')]]

# Word Sense Disambiguation

In [15]:
from nltk.wsd import lesk

x = 'The porn threatens to cause a lockdown'

print(lesk(x.split(), 'threatens', 'v'))


Synset('threaten.v.03')


In [16]:
def word_definitions(title, lemmed_title):
    synsets = []
    original_text = title
    lemmed_pos = lemmed_title
    for i in lemmed_pos:
        synsets.append(lesk(original_text.split(), i[0], i[1]))
    return synsets

In [17]:
x = newspapers['title'][1]
y = newspapers['lem_pos_headline'][1]
import re
from nltk.corpus import sentiwordnet as swn,SentiSynset

j = word_definitions(x, y)
k = str(j[0])
print(k)
x = (re.findall(r"'([^']*)'", k))
print(x)
f = swn.senti_synset(x[0])
print(f)

Synset('agnosticism.n.02')
['agnosticism.n.02']
<agnosticism.n.02: PosScore=0.25 NegScore=0.0>


In [18]:
newspapers['word_definitions'] = newspapers.apply(lambda x: word_definitions(x['title'], x['lem_pos_headline']), axis=1)


In [19]:
x = newspapers['word_definitions'][1]

In [20]:
for i in x:
    syn_word = (re.findall(r"'([^']*)'", str(i)))
    if len(syn_word) != 0:
        input_word = (syn_word[0])
        word_senti = (swn.senti_synset(input_word))
        pos = float(word_senti.pos_score()) 
        neg = float(word_senti.neg_score())
        overall_senti = pos + neg
        print(overall_senti)

0.25
0.25
0.0
0.0
0.0


# TF − IDF ∗ SentimentScore

In [21]:
newspapers

,hrefs,title,new_title,id,date,newspaper_name,vaccine,pos_headline,lem_pos_headline,word_definitions
0,https://www.theguardian.com/politics/2021/mar/...,Boris Johnson receives Oxford/AstraZeneca Covi...,bor johnson receiv oxfordastrazenec covid vaccin,politics,2021-03-19,guardian,NaN,"[[(Boris, NNP)], [(Johnson, NNP)], [(receives,...","[[receives, n], [Covid, n], [vaccine, n]]","[None, None, Synset('vaccine.n.01')]"
1,https://www.theguardian.com/world/2021/feb/19/...,Scepticism over Oxford vaccine threatens Europ...,sceiv oxford vaccin threatens europ immun push,world,2021-02-19,guardian,NaN,"[[(Scepticism, NN)], [(over, IN)], [(Oxford, N...","[[Scepticism, n], [Oxford, n], [vaccine, n], [...","[Synset('agnosticism.n.02'), Synset('oxford.n...."
2,https://www.theguardian.com/world/2021/mar/18/...,Thursday briefing: EU's experts to give Oxford...,thursday brief eu expert giv oxford vaccin ver...,world,2021-03-18,guardian,NaN,"[[(Thursday, NNP)], [(experts, NNS)], [(to, TO...","[[expert, n], [give, v], [Oxford, n], [vaccine...","[Synset('expert.n.01'), Synset('give.v.18'), S..."
3,https://www.theguardian.com/world/2021/mar/16/...,Oxford/AstraZeneca vaccine: which countries ha...,oxfordastrazenec vaccin country paus jab,world,2021-03-16,guardian,NaN,"[[(which, WDT)], [(countries, NNS)], [(have, V...","[[country, n], [have, v], [pause, v], [jab, n]]","[Synset('country.n.04'), Synset('take.v.35'), ..."
4,https://www.theguardian.com/world/2021/mar/16/...,Chaos in Germany and Italy after suspension of...,chao germany ita suspend oxford vaccin,world,2021-03-16,guardian,NaN,"[[(Chaos, NN)], [(in, IN)], [(Germany, NNP)], ...","[[Chaos, n], [suspension, n], [Oxford, n], [va...","[Synset('chaos.n.02'), Synset('suspension.n.05..."
...,...,...,...,...,...,...,...,...,...,...
62019,https://www.independent.co.uk/news/uk/politics...,Extreme lockdown laws extended for a further s...,extrem lockdown law extend six month despit ma...,coronavirus,2021-03-25,independent,NaN,"[[(Extreme, NN)], [(lockdown, NN)], [(laws, NN...","[[Extreme, n], [lockdown, n], [law, n], [exten...","[Synset('extreme_point.n.01'), Synset('lockdow..."
62020,https://www.independent.co.uk/news/uk/politics...,Extreme lockdown laws extended for a further s...,extrem lockdown law extend six month despit ma...,coronavirus,2021-03-25,independent,NaN,"[[(Extreme, NN)], [(lockdown, NN)], [(laws, NN...","[[Extreme, n], [lockdown, n], [law, n], [exten...","[Synset('extreme_point.n.01'), Synset('lockdow..."
62021,https://www.independent.co.uk/independentpremi...,The problem with Johnson’s vaccine passports f...,the problem johnson vaccin passport pub,business,2021-03-25,independent,NaN,"[[(The, DT)], [(problem, NN)], [(with, IN)], [...","[[problem, n], [vaccine, n], [passport, n], [p...","[Synset('problem.n.02'), Synset('vaccine.n.01'..."
62022,https://www.independent.co.uk/voices/editorial...,Reality is dawning on the EU: export bans are ...,real dawn eu export ban counterproduc cas covi...,uk,2021-03-25,independent,NaN,"[[(Reality, NN)], [(is, VBZ)], [(dawning, VBG)...","[[Reality, n], [be, v], [dawn, v], [export, n]...","[Synset('reality.n.03'), Synset('embody.v.02')..."


In [22]:
N = len (newspapers)

In [23]:
DF = {}
processed_text = newspapers.new_title.tolist()
for i in range(len(processed_text)):
    tokens = processed_text[i]
    local = tokens.split(' ')
    for w in local:
        try:
            DF[w].add(i)
        except:
            DF[w] = {i}


In [24]:
for i in DF:
    DF[i] = len(DF[i])

In [25]:
total_vocab = [x for x in DF]
print(total_vocab)

['bor', 'johnson', 'receiv', 'oxfordastrazenec', 'covid', 'vaccin', 'sceiv', 'oxford', 'threatens', 'europ', 'immun', 'push', 'thursday', 'brief', 'eu', 'expert', 'giv', 'verdict', 'country', 'paus', 'jab', 'chao', 'germany', 'ita', 'suspend', 'caut', 'sci', 'frant', 'press', 'shift', 'rollout', 'plan', 'chang', 'follow', 'approv', 'belg', 'consid', 'uturn', 'over55s', 'slow', 'distribut', 'dos', 'cas', 'ris', 'debenham', 'shut', 'six', 'stor', 'includ', 'street', 'flag', 'spat', 'could', 'lead', 'diverg', 'strategies', 'what', 'diff', 'mak', 'uk', 'ful', 'reg', 'ms', 'redevelop', 'big', 'shop', 'mov', 'onlin', 'quest', 'hang', 'undergo', 'new', 'glob', 'tri', 'lif', 'sav', 'amaz', 'story', 'stil', 'review', 'say', 'medicin', 'ag', 'hancock', 'seiz', 'dat', 'count', 'doubt', 'tuesday', 'success', 'boost', 'hop', 'market', 'begin', '', 'busy', 'liv', 'hospit', 'monday', '10', 'eff', 'sou', 'afr', 'vary', 'study', 'suggest', 'why', 'adv', 'over65s', 'show', 'less', 'protect', 'how', 'dep

In [26]:
def doc_freq(word):
    c = 0
    try:
        c = DF[word]
    except:
        pass
    return c

In [27]:
processed_text[1]

'sceiv oxford vaccin threatens europ immun push'

In [28]:
N = len(newspapers)

In [29]:
doc = 0
tf_idf = {}
for i in range(1):
    headline = processed_text[i]
    new_headline = headline.split(' ')
    counter = Counter(new_headline)
    words_count = len(new_headline)
    for token in np.unique(new_headline):
        tf = counter[token]/words_count
        df = doc_freq(token)
        idf = np.log((N+1)/(df+1))
        tf_idf[doc, token] = tf*idf
        print(doc, tf*idf)
    doc += 1

0 0.4895430087306552
0 0.38863100651337873
0 0.4776122078060252
0 1.194015299608685
0 0.938769237781287
0 0.4599011898476075


In [30]:
tf_idf

{(0, 'bor'): 0.4895430087306552,
 (0, 'covid'): 0.38863100651337873,
 (0, 'johnson'): 0.4776122078060252,
 (0, 'oxfordastrazenec'): 1.194015299608685,
 (0, 'receiv'): 0.938769237781287,
 (0, 'vaccin'): 0.4599011898476075}

In [31]:
test=newspapers.reset_index(drop=False, inplace=False)
df_model2=test

In [32]:
index_df=list(df_model2.index)


In [33]:
N = len (df_model2)

sum_sentiment={}

for index in index_df:
    headline = processed_text[index]
    new_headline = headline.split(' ')
    #print(new_headline)
    for token in np.unique(new_headline):
        #print(index)
        #print(token)
        if index in list(sum_sentiment.keys()):
            sentiment_value=float(tf_idf[(index, token)])*float(tf_idf[(index, token)])
            sum_sentiment[index] += sentiment_value
        else:
            sum_sentiment[index]=float(tf_idf[(index, token)])*float(tf_idf[(index, token)])

KeyError: (1, 'europ')

In [ ]:
sum_sentiment

In [ ]:
df_model2["sentiment"]=df_model2.apply(lambda row: sum_sentiment[row.name],axis=1)

In [ ]:
df_model2